In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkConf,SparkContext

In [6]:
conf = SparkConf().setMaster("local").setAppName("KeyValueRDD")
sc = SparkContext(conf=conf)

In [10]:
# rdd.map(lambda x : (x,1))  -> this is how key value RDD is formed

# rdd.reduceByKey(lambda x,y : x+y)   --> it is an action
# groupByKey()
# sortByKey()
# rdd.values()  -> to get the rdd of only values
# rdd.keys()  -> to get the rdd of only keys


# SQL style joins on 2 key-value RDDs
# join , rightOuterJoin , leftOuterJoin , cogroup , subtractByKey

# With key-value data,use mapValues() and flatMapValues() if your transformation doesn't affect the keys i.e when keys
# are not being modified

In [11]:
rdd = sc.textFile("file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files/fakefriends.csv")

In [36]:
print(type(rdd))

<class 'pyspark.rdd.RDD'>


In [37]:
def split_line(line):
    info = line.split(',')
    age = int(info[2])
    friends = int(info[3])
    return (age,friends)

In [38]:
ageFriendsRDD = rdd.map(split_line)  # (age,number of friends) , one value means one line

In [56]:
print(type(ageFriendsRDD))

<class 'pyspark.rdd.PipelinedRDD'>


In [57]:
# print(ageFriendsRDD.collect())

In [58]:
# groupedRDD = ageFriendsRDD.groupByKey()

In [59]:
# print(groupedRDD.collect())

In [60]:
modifiedAgeFriendsRDD = ageFriendsRDD.mapValues(lambda x:(x,1)) # Here you will get x as just values, not keys because
# mapValues() function is used on rdd instead of map()

In [61]:
reducedRDD = modifiedAgeFriendsRDD.reduceByKey(lambda x,y : (x[0]+y[0],x[1]+y[1]))  
# So here you would get x and y both as tuples
# in reduceByKey(lambda x,y: ...do something...) --> here in lambda function, values are passed only(not keys) 

In [76]:
print(reducedRDD.collect())
print(type(reducedRDD.collect()))

In [66]:
print(reducedRDD.count())
print(type(reducedRDD.count()))

52
<class 'int'>


In [70]:
finalRDD = reducedRDD.mapValues(lambda x:x[0]/x[1])

In [75]:
print(finalRDD.collect())

In [74]:
results = finalRDD.collect()

for result in results:
    print(result)

In [ ]:
# END